In [574]:
import pandas as pd   
import warnings; warnings.simplefilter('ignore')
import math

## Integer Programming: Cutting Plane Method

In [575]:


objective="min"
m=2#no of equations
n=3#no of variables
a=[[1,4,2,'>='],[3,1,2,">="]]
b=[5,4]
Coeff=[2,9,1]

In [576]:
def convert_to_maximization_type_phase1(a,b,Coeff,m,n,objective):    
        for i in range(m):
            if b[i] < 0:
                for j in range(n):
                    a[i][j]=-a[i][j]
                if a[i][-1]=="<=":
                    a[i][-1]=">="
                elif a[i][-1]==">=":
                    a[i][-1]="<="
                    
                b[i]=-b[i]
        return a,b,m,n
                
def convert_to_maximization_type_phase2(Coeff):
    if objective != 'max':
            Coeff=[-Coeff[i] for i in range(n)]
            return Coeff
    else:
        return Coeff
    
                
    

In [577]:


#1
def initialize_phase1_simplex_table(a, b, m, n, Coeff,objective):
    data={}
    col=['x'+str(i+1) for i in range(n)]
    convert_dict={'x'+str(i+1):float for i in range(n)}
    jconvert_dict={'x'+str(i+1):float for i in range(n)}
    a,b,m,n=convert_to_maximization_type_phase1(a,b,Coeff,m,n,objective)
    Cj_dict={'x'+str(i+1):[0] for i in range(n)}
    Zj_dict={'x'+str(i+1):[0] for i in range(n)}
    
    for i in range(m):
        if a[i][-1] == "<=":
            Cj_dict['sl'+str(i+1)]=[0]
            Zj_dict['sl'+str(i+1)]=[0]
            convert_dict['sl'+str(i+1)]=float
            jconvert_dict['sl'+str(i+1)]=float
            col.append('sl'+str(i+1))
        elif a[i][-1] == "=":
            Cj_dict['a'+str(i+1)]=[-1]
            Zj_dict['a'+str(i+1)]=[0]
            convert_dict['a'+str(i+1)]=float
            jconvert_dict['a'+str(i+1)]=float
            col.append('a'+str(i+1))
        else:
            Cj_dict['sr'+str(i+1)]=[0]
            Zj_dict['sr'+str(i+1)]=[0]
            convert_dict['sr'+str(i+1)]=float
            jconvert_dict['sr'+str(i+1)]=float
            col.append('sr'+str(i+1))
            Cj_dict['a'+str(i+1)]=[-1]
            Zj_dict['a'+str(i+1)]=[0]
            convert_dict['a'+str(i+1)]=float
            jconvert_dict['a'+str(i+1)]=float
            col.append('a'+str(i+1))
            
    Cj_dict['sol']=0
    Zj_dict['sol']=0   
    convert_dict['sol']=float
    jconvert_dict['sol']=float
    #Cj_dict.extend({0 for i in range(m)])
    #Zj=[0 for i in range(m+n)]
    data['CBi']=[]    
    convert_dict['CBi']=float    
    data['basic_var']=[]    
    convert_dict['basic_var']=str
    data['sol']=[b[j] for j in range(m)]
    data['ratio']=[0 for j in range(m)]
    for i in range(n):
        data['x'+str(i+1)]=[a[j][i] for j in range(m)]
    #augmenting data with slack variables, surplus variables and artificial variables
    for i in range(m):
        if a[i][-1] == "<=":
            data['sl'+str(i+1)]=[float(i==j) for j in range(m)]
            data['basic_var'].append('sl'+str(i+1))
            data['CBi'].append(0)
        elif a[i][-1] == "=":
            data['a'+str(i+1)]=[float(i==j) for j in range(m)]
            data['basic_var'].append('a'+str(i+1))
            data['CBi'].append(-1)
        else:
            data['sr'+str(i+1)]=[-1*float(i==j) for j in range(m)]
            data['a'+str(i+1)]=[float(i==j) for j in range(m)]
            data['basic_var'].append('a'+str(i+1))
            data['CBi'].append(-1)
        
    
    convert_dict['ratio']=float
    df = pd.DataFrame(data, index=[str(i+1) for i in range(m)]) 
    Zj = pd.DataFrame(Zj_dict, index=['Zj']) 
    row=list(df.index)
    #col.append('sol')
    for j in col:
        Zj[j]['Zj']=sum([df[j][i]*df['CBi'][i] for i in row]) 
    Zj['sol']['Zj']=sum([df['sol'][i]*df['CBi'][i] for i in row])
    Cj = pd.DataFrame(Cj_dict, index=['Cj'])    
    df = df.astype(convert_dict)     
    Zj = Zj.astype(jconvert_dict)
    Cj = Cj.astype(jconvert_dict)
    return df,Cj,Zj,col
  

In [578]:
#2 
def print_table(df, Cj, Zj):
    print("simplex table:")
    frames = [df, Cj, Zj]
    result = pd.concat(frames)    
    print(result)

    
  
 
# df,Cj,Zj,col = initialize_phase1_simplex_table(a, b, m, n, Coeff,objective)
# print_table(df,Cj,Zj)

In [579]:
#3

def Zj_Cj(Cj, Zj,col):    
    return  {i:Zj[i]['Zj']-Cj[i]['Cj'] for i in col}


In [580]:
#4
# def col_name(i,n):
#     if (i+1)<=n:
#         return 'x'+str(i+1)
#     else:
#         return 's'+str(i+1-n)
    
def get_key_column(zj_cj,df, m, n,col):
    val_min=9999999999
    idx_min=None
    for i in (set(col)-set(df['basic_var']))-{'sol'}:
        if val_min>zj_cj[i]:
            val_min=zj_cj[i]
            idx_min=i
    
    return idx_min
    


def get_key_row(df, key_col, m):   
    df['ratio']=df['sol']/df[key_col]
    row=list(df.index)
    val_min=999999999
    idx_min='1'
    for i in row:
        if val_min>df['ratio'][i]>=0:
            val_min=df['ratio'][i]
            idx_min=i
    return df, idx_min, val_min




In [581]:
def modify_table(key_row, key_col, Cj, Zj,df,col):
    df_new = df.copy(deep=True)
    Zj_new = Zj.copy(deep=True)
    cols=col[:]
    cols.append('sol')
    col_len=len(df[key_col])
    row_len=len(df.loc[key_row])
    p=key_elem=df[key_col][key_row]
    entering_var=key_col
    leaving_var=df['basic_var'][key_row]
    print("Entering variable is ", entering_var)
    print("Leaving variable is ", leaving_var)
    df_new['basic_var'][key_row]=entering_var
    #swap the coefficients value
    df_new['CBi'][key_row]=Cj[key_col]['Cj']    
    row=list(df.index)    
    for j in cols:
        r=df[j][key_row]              
        for i in row:    
            if j==key_col:
                df_new[j][i]=(-df[j][i])/p                
            elif i==key_row:
                df_new[j][i]=df[j][i]/p               
            else:
                s=df[j][i]
                q=df[key_col][i] 
                df_new[j][i]=(p*s-q*r)/p
    #print(1/p)
    df_new[key_col][key_row]=1/p
    #df_new[key_col]=df[]
    df_new[leaving_var]  = df_new[entering_var]    
    df_new[entering_var] = df[leaving_var]
    #df_new[entering_var][key_row]=1
    #modify zj
    for j in cols:
        Zj_new[j]['Zj']=sum([df_new[j][i]*df_new['CBi'][i] for i in row])
    
        #Zj_new[j]['Zj']=16.0*0.05
    return df_new,Cj,Zj_new
    
    
    
    
    
    
    
    
    

In [582]:
# df_new,Cj,Zj_new=modify_table(key_row, key_col, Cj, Zj,df,col)
# print_table(df_new,Cj,Zj_new)


In [583]:
#Check is optimal solution is reached or not
def is_optimal(zj_cj,df):
#     zj_cj_soless=zj_cj.copy()
    #del zj_cj_soless['sol']
    val=min( [value for (key,value) in zj_cj.items() if key not in set(df['basic_var']).union({'sol'})])
#     val=min(zj_cj_soless.values())
    if val>=0:
        print("Optimality is reached")
        return True
    else:
        print("Lets proceed to next iteration")
        return False

In [584]:
def phase1_simplex_method(a, b, m, n, Coeff,objective):    
    df,Cj,Zj,col = initialize_phase1_simplex_table(a, b, m, n, Coeff,"min")      
    zj_cj=Zj_Cj(Cj, Zj,col)
    key_col=get_key_column(zj_cj,df, m, n,col)
    df, key_row, min_ratio = get_key_row(df, key_col, m)
    print_table(df,Cj,Zj)
    if objective=="max":
        print("value of objective function is ",Zj['sol']['Zj'])
    else:
        print("value of objective function is ",-Zj['sol']['Zj'])
    i=0
    while not is_optimal(zj_cj,df) and i<10:
        print("_______________________________________________________")
        if df[key_col][key_row]==0:
            print("Problem is unbounded")
            return None, None, None, None
        df_new,Cj,Zj_new=modify_table(key_row, key_col, Cj, Zj,df,col)
        
        df=df_new
        Zj=Zj_new
        print_table(df,Cj,Zj)        
        zj_cj=Zj_Cj(Cj, Zj,col)
        key_col=get_key_column(zj_cj,df, m, n,col)
        df, key_row, min_ratio = get_key_row(df, key_col, m) 
        if objective=="max":
            print("value of objective function is ",Zj['sol']['Zj'])
        else:
            print("value of objective function is ",-Zj['sol']['Zj'])
            
        i+=1
        
    
    return Zj['sol']['Zj'], df, Cj, Zj
        
    
    

In [585]:
#Check if we can proceed to phase 2 
def can_go_to_phase2(optimal_val):
    if optimal_val==0:
        return True
    else:
        return False

In [586]:
#Initialize phase 2 simplex table
def initialize_phase2_simplex_table(df, Cj, Zj, Coeff, m,n, objective):
    #drop all artificial variables
    Coeff = convert_to_maximization_type_phase2(Coeff)
    artificial_var=[]
    for i in range(1, m+1):
        if "a"+str(i) in df:
            artificial_var.append("a"+str(i))
    df.drop(artificial_var, axis = 1, inplace = True)   
    Cj.drop(artificial_var, axis = 1, inplace = True) 
    Zj.drop(artificial_var, axis = 1, inplace = True) 
    
    #initialize Cj's
    for i in range(1,n+1):
        Cj["x"+str(i)]=Coeff[i-1]
       
    #Modify CBi's
    for i in range(1,m+1):
        df['CBi'][str(i)]=Cj[df['basic_var'][str(i)]]
        
    #re-calculate Zj's
    row=list(df.index)
    col=list (df.columns)
    col.remove('CBi')
    col.remove('basic_var')
    col.remove('ratio')    
    for j in col:
        Zj[j]['Zj']=sum([df[j][i]*df['CBi'][i] for i in row]) 
    return df, Cj, Zj, col

In [587]:
def phase2_simplex_method(df, Cj, Zj, Coeff, m,n,objective):     
    df,Cj,Zj,col = initialize_phase2_simplex_table(df, Cj, Zj, Coeff, m,n,objective)
    zj_cj=Zj_Cj(Cj, Zj,col)
    key_col=get_key_column(zj_cj,df, m, n,col)
    df, key_row, min_ratio = get_key_row(df, key_col, m)
    print_table(df,Cj,Zj)
    if objective=="max":
        print("value of objective function is ",Zj['sol']['Zj'])
    else:
        print("value of objective function is ",-Zj['sol']['Zj'])
    i=0
    while not is_optimal(zj_cj,df) and i<10:
        print("_______________________________________________________")
        if df[key_col][key_row]==0:
            print("Problem is unbounded")
            return 
        df_new,Cj,Zj_new=modify_table(key_row, key_col, Cj, Zj,df,col)
        
        df=df_new
        Zj=Zj_new
        print_table(df,Cj,Zj)        
        zj_cj=Zj_Cj(Cj, Zj,col)
        key_col=get_key_column(zj_cj,df, m, n,col)
        df, key_row, min_ratio = get_key_row(df, key_col, m) 
        if objective=="max":
            print("value of objective function is ",Zj['sol']['Zj'])
        else:
            print("value of objective function is ",-Zj['sol']['Zj'])
            
        i+=1
        
    
    return df, Cj, Zj, col,m,n
        
    

In [588]:
def two_phase_simplex(a, b, m, n, Coeff,objective):
    print("*********************************Phase 1************************************************")
    optimal_val, df, Cj, Zj = phase1_simplex_method(a, b, m, n, Coeff,objective)

    if optimal_val is not None:
        if can_go_to_phase2(optimal_val):
            print("Value of the objective function is zero, let's proceed to phase2 of simplex method")
            print("*********************************Phase 2************************************************")
            try:
                df,Cj,Zj,col,m,n=phase2_simplex_method(df, Cj, Zj, Coeff, m,n,objective)
                return df,Cj,Zj,col,m,n
            except:
                return
        else:
            print("Since all artificial variables can't be made zero, we can't proceed to phase2. Hence the solution is infeasible")

### adding cutting plane equuation

In [589]:
def f_part(num):
    if num>0:
        return num-int(num)
    else:
        return num+1-int(num)

In [590]:
def largest_fractional_row(df):
    val_max=-1
    val_idx=None
    for idx in df.index:
        f=f_part(df['sol'][idx])
        if f>=val_max:
            val_max=f
            val_idx=idx
    #print("------",val_idx,val_max)
    return val_idx, val_max
        

In [591]:
def add_cutting_equation(df,Cj,Zj,idx,col,m,n):
    d={'CBi':0,'basic_var':'sl'+str(m+1),'ratio':0,'sl'+str(m+1):1.000000}
    for val in col:          
        d[val]=-f_part(df[val][idx])
    d['sol']=-f_part(df['sol'][idx])
    df['sl'+str(m+1)]=0.000000
    Cj['sl'+str(m+1)]=0.000000
    Zj['sl'+str(m+1)]=0.000000    
    new_row = pd.DataFrame(d, index=[str(m+1)])  
    frames = [df, new_row]
    df = pd.concat(frames)
    col.append('sl'+str(m+1))
    return df, Cj, Zj, col, m+1, n
    
            

### Dual Simplex Method

In [592]:


# def col_name(i,n):
#     if (i+1)<=n:
#         return 'x'+str(i+1)
#     else:
#         return 's'+str(i+1-n)
    
def get_key_column_dual(zj_cj,df,col, m, n, key_row):
    val_max=-999999999
    idx_max='x1'  
    for c_name in (set(col)-set(df['basic_var']))-{'sol'}:
        #c_name=col_name(i,n)
        ratio=zj_cj[c_name]/df[c_name][key_row]
        if df[c_name][key_row]<0:
            if val_max<ratio:
                val_max=ratio
                idx_max=c_name
    
    return idx_max
    


def get_key_row_dual(df, m):     
    val_min=0
    idx_min=0
    for i in range(m):
        if val_min>df['sol'][i]:
            val_min=df['sol'][i]
            idx_min=i
    return df, str(idx_min+1), val_min


In [593]:
def modify_table_dual(key_row, key_col, Cj, Zj,df,col):
    df_new = df.copy(deep=True)
    Zj_new = Zj.copy(deep=True)
    cols=col[:]
    cols.append('sol')
    col_len=len(df[key_col])
    row_len=len(df.loc[key_row])
    p=key_elem=df[key_col][key_row]
    entering_var=key_col
    leaving_var=df['basic_var'][key_row]
    print("Entering variable is ", entering_var)
    print("Leaving variable is ", leaving_var)
    df_new['basic_var'][key_row]=entering_var
    
    
#     col=['x'+str(i+1) for i in range(n)]
#     col.extend(['s'+str(i+1) for i in range(m)])
#     col.append('sol')
    row=list(df.index)
    
    for j in cols:
        r=df[j][key_row]              
        for i in row:    
            if j==key_col:
                df_new[j][i]=(-df[j][i])/p                
            elif i==key_row:
                df_new[j][i]=df[j][i]/p               
            else:
                s=df[j][i]
                q=df[key_col][i] 
                df_new[j][i]=(p*s-q*r)/p
    #print(1/p)
    df_new[key_col][key_row]=1/p
    #df_new[key_col]=df[]
    df_new[leaving_var]  = df_new[entering_var]    
    df_new[entering_var] = 0
    df_new[entering_var][key_row]=1
    
    #swap the coefficients value
    df_new['CBi'][key_row]=Cj[key_col]['Cj']
    
    #modify zj
    for j in cols:
        Zj_new[j]['Zj']=sum([df_new[j][i]*df_new['CBi'][i] for i in row]) 
        #Zj_new[j]['Zj']=16.0*0.05
    return df_new,Cj,Zj_new
    
    

In [594]:
#Check is feasible solution is reached or not
def is_feasible(df):
    for val in df['sol']:
        if val<0:
            print("Lets proceed to next iteration")
            return False
    else:   
        print("Feasibility is reached")
        return True
    

In [595]:
def dual_simplex_method(df,Cj,Zj,col, m, n, objective):    
    #df,Cj,Zj = initialize_simplex_table(a, b, m, n, Coeff,objective)
    zj_cj=Zj_Cj(Cj, Zj,col)
    df, key_row, min_ratio = get_key_row_dual(df, m)
    key_col=get_key_column_dual(zj_cj,df,col, m, n,key_row)
    print_table(df,Cj,Zj)
    i=0
    while not is_feasible(df) and i<10:
        print("_______________________________________________________")
        if df[key_col][key_row]==0:
            print("Problem is unbounded")
            return
        df_new,Cj,Zj_new=modify_table_dual(key_row, key_col, Cj, Zj,df,col)
        
        df=df_new
        Zj=Zj_new
        
        if objective=="max":
            print("value of objective function is ",Zj['sol']['Zj'])
        else:
            print("value of objective function is ",-Zj['sol']['Zj'])
        
        zj_cj=Zj_Cj(Cj, Zj,col)
        df, key_row, min_ratio = get_key_row_dual(df, m)
        key_col=get_key_column_dual(zj_cj,df,col, m, n,key_row)  
        print_table(df,Cj,Zj)
        i+=1
    return df,Cj,Zj,m,n,col
        
    
    

In [596]:
def cutting_plane_method(a, b, m, n, Coeff,objective):
    #try:
    df,Cj,Zj,col,m,n=two_phase_simplex(a, b, m, n, Coeff,objective)
    for i in range(10):
        idx,val_max=largest_fractional_row(df)
        if val_max<0.0001 or val_max>0.9999:
            print("Found integer solutions")   
            print("-----------basic variables and their values-----------------------")
            print(df[['basic_var','sol']])
            print("------------------------------------------------------------------")
            break
        else:
            print("solutions are not integers, adding cutting plane equation")
        df,Cj,Zj,col,m,n=add_cutting_equation(df,Cj,Zj,idx,col,m,n)        
        df,Cj,Zj,m,n,col=dual_simplex_method(df,Cj,Zj,col, m, n, objective)  
        

#     except:
#         print("Terminating the process......")
#         return
    

In [597]:
#1
objective="min"
m=4#no of equations
n=3#no of variables
a=[[-1,2,-6,'<='],[0,1,2,"<="],[2,0,10,"<="],[-1,1,0,"<="]]
b=[-10,6,19,-2]
Coeff=[2,15,18]

cutting_plane_method(a, b, m, n, Coeff,objective)

*********************************Phase 1************************************************
simplex table:
    CBi basic_var   sol     ratio   x1   x2    x3  sr1   a1  sl2  sl3  sr4  \
1  -1.0        a1  10.0  1.666667  1.0 -2.0   6.0 -1.0  1.0  0.0  0.0 -0.0   
2   0.0       sl2   6.0  3.000000  0.0  1.0   2.0 -0.0  0.0  1.0  0.0 -0.0   
3   0.0       sl3  19.0  1.900000  2.0  0.0  10.0 -0.0  0.0  0.0  1.0 -0.0   
4  -1.0        a4   2.0       inf  1.0 -1.0   0.0 -0.0  0.0  0.0  0.0 -1.0   
Cj  NaN       NaN   0.0       NaN  0.0  0.0   0.0  0.0 -1.0  0.0  0.0  0.0   
Zj  NaN       NaN -12.0       NaN -2.0  3.0  -6.0  1.0 -1.0  0.0  0.0  1.0   

     a4  
1   0.0  
2   0.0  
3   0.0  
4   1.0  
Cj -1.0  
Zj -1.0  
value of objective function is  12.0
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x3
Leaving variable is  a1
simplex table:
    CBi basic_var       sol     ratio        x1        x2   x3       sr1  \
1   0.0        

In [598]:
#2
objective="max"
m=2#no of equations
n=2#no of variables
a=[[3,-1,'<='],[3,11,"<="]]
b=[12,66]
Coeff=[3,4]

cutting_plane_method(a, b, m, n, Coeff,objective)

*********************************Phase 1************************************************
simplex table:
    CBi basic_var   sol  ratio   x1    x2  sl1  sl2
1   0.0       sl1  12.0  -12.0  3.0  -1.0  1.0  0.0
2   0.0       sl2  66.0    6.0  3.0  11.0  0.0  1.0
Cj  NaN       NaN   0.0    NaN  0.0   0.0  0.0  0.0
Zj  NaN       NaN   0.0    NaN  0.0   0.0  0.0  0.0
value of objective function is  0.0
Optimality is reached
Value of the objective function is zero, let's proceed to phase2 of simplex method
*********************************Phase 2************************************************
simplex table:
    CBi basic_var   sol  ratio   x1    x2  sl1  sl2
1   0.0       sl1  12.0  -12.0  3.0  -1.0  1.0  0.0
2   0.0       sl2  66.0    6.0  3.0  11.0  0.0  1.0
Cj  NaN       NaN   0.0    NaN  3.0   4.0  0.0  0.0
Zj  NaN       NaN   0.0    NaN  0.0   0.0  0.0  0.0
value of objective function is  0.0
Lets proceed to next iteration
_______________________________________________________
Entering

In [599]:
#3
objective="max"
m=3#no of equations
n=2#no of variables
a=[[0,2,'<='],[1,1,"<="],[2,0,">="]]
b=[7,7,11]
Coeff=[1,1]

cutting_plane_method(a, b, m, n, Coeff,objective)

*********************************Phase 1************************************************
simplex table:
    CBi basic_var   sol  ratio   x1   x2  sl1  sl2  sr3   a3
1   0.0       sl1   7.0    inf  0.0  2.0  1.0  0.0 -0.0  0.0
2   0.0       sl2   7.0    7.0  1.0  1.0  0.0  1.0 -0.0  0.0
3  -1.0        a3  11.0    5.5  2.0  0.0  0.0  0.0 -1.0  1.0
Cj  NaN       NaN   0.0    NaN  0.0  0.0  0.0  0.0  0.0 -1.0
Zj  NaN       NaN -11.0    NaN -2.0  0.0  0.0  0.0  1.0 -1.0
value of objective function is  -11.0
Lets proceed to next iteration
_______________________________________________________
Entering variable is  x1
Leaving variable is  a3
simplex table:
    CBi basic_var  sol  ratio   x1   x2  sl1  sl2  sr3   a3
1   0.0       sl1  7.0    inf  0.0  2.0  1.0  0.0  0.0 -0.0
2   0.0       sl2  1.5    7.0  0.0  1.0  0.0  1.0  0.5 -0.5
3   0.0        x1  5.5    5.5  1.0  0.0  0.0  0.0 -0.5  0.5
Cj  NaN       NaN  0.0    NaN  0.0  0.0  0.0  0.0  0.0 -1.0
Zj  NaN       NaN  0.0    NaN  0.0  0.0  

In [600]:
#4
objective="max"
m=2#no of equations
n=2#no of variables
a=[[-1,3,'<='],[7,1,"<="]]
b=[6,35]
Coeff=[7,9]

cutting_plane_method(a, b, m, n, Coeff,objective)

*********************************Phase 1************************************************
simplex table:
    CBi basic_var   sol  ratio   x1   x2  sl1  sl2
1   0.0       sl1   6.0    2.0 -1.0  3.0  1.0  0.0
2   0.0       sl2  35.0   35.0  7.0  1.0  0.0  1.0
Cj  NaN       NaN   0.0    NaN  0.0  0.0  0.0  0.0
Zj  NaN       NaN   0.0    NaN  0.0  0.0  0.0  0.0
value of objective function is  0.0
Optimality is reached
Value of the objective function is zero, let's proceed to phase2 of simplex method
*********************************Phase 2************************************************
simplex table:
    CBi basic_var   sol  ratio   x1   x2  sl1  sl2
1   0.0       sl1   6.0    2.0 -1.0  3.0  1.0  0.0
2   0.0       sl2  35.0   35.0  7.0  1.0  0.0  1.0
Cj  NaN       NaN   0.0    NaN  7.0  9.0  0.0  0.0
Zj  NaN       NaN   0.0    NaN  0.0  0.0  0.0  0.0
value of objective function is  0.0
Lets proceed to next iteration
_______________________________________________________
Entering variable 